### Ejercicio 7 :
- [x] Generar un DataFrame agrupado por edad
- [x] Mostrar las descriptivas del DataFrame
- [x] Generar una tabla pivote a partir del DataFrame
- [x] Generar 2 DataFrame
- [x] Unir los 2 DataFrame tipo interno
- [x] Unir los 2 DataFrame tipo externo
- [x] Unir los 2 DataFrame natural

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, count

spark = SparkSession.builder \
    .appName("Exercise7") \
    .getOrCreate()


24/07/26 12:55:31 WARN Utils: Your hostname, Macbook-Air-31.local resolves to a loopback address: 127.0.0.1; using 192.168.68.102 instead (on interface en0)
24/07/26 12:55:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/26 12:55:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
schema = "Name STRING, Age INT, Gender STRING"

data = [
    ("David", 19, "M"),
    ("Mark", 25, "M"),
    ("Shirley", 22, "F"),
    ("Jasmine", 50, "F")
]

df = spark.createDataFrame(data, schema)

In [3]:
df_grouped = df.groupBy('Age').count()
print("Grouped by Age:")
df_grouped.show()

Grouped by Age:


+---+-----+
|Age|count|
+---+-----+
| 19|    1|
| 25|    1|
| 22|    1|
| 50|    1|
+---+-----+



In [4]:
print("DF descriptives")
df_grouped.describe().show()

DF descriptives
+-------+------------------+-----+
|summary|               Age|count|
+-------+------------------+-----+
|  count|                 4|    4|
|   mean|              29.0|  1.0|
| stddev|14.212670403551895|  0.0|
|    min|                19|    1|
|    max|                50|    1|
+-------+------------------+-----+



In [5]:
df_pivot = df.groupBy("Gender").pivot("Age").count()
print("Pivot Table:")
df_pivot.show()

Pivot Table:
+------+----+----+----+----+
|Gender|  19|  22|  25|  50|
+------+----+----+----+----+
|     F|NULL|   1|NULL|   1|
|     M|   1|NULL|   1|NULL|
+------+----+----+----+----+



In [6]:
data1 = [
    ("David", 19, "M"),
    ("Mark", 25, "M"),
    ("Shirley", 22, "F")
]

data2 = [
    ("Jasmine", 50, "F"),
    ("John", 35, "M"),
    ("Shirley", 22, "F")
]

df1 = spark.createDataFrame(data1, schema)
df2 = spark.createDataFrame(data2, schema)

print("DataFrame 1:")
df1.show()

print("DataFrame 2:")
df2.show()

DataFrame 1:
+-------+---+------+
|   Name|Age|Gender|
+-------+---+------+
|  David| 19|     M|
|   Mark| 25|     M|
|Shirley| 22|     F|
+-------+---+------+

DataFrame 2:
+-------+---+------+
|   Name|Age|Gender|
+-------+---+------+
|Jasmine| 50|     F|
|   John| 35|     M|
|Shirley| 22|     F|
+-------+---+------+



In [7]:
df_inner = df1.join(df2, on="Name", how="inner")
print("Inner join:")
df_inner.show()

Inner join:
+-------+---+------+---+------+
|   Name|Age|Gender|Age|Gender|
+-------+---+------+---+------+
|Shirley| 22|     F| 22|     F|
+-------+---+------+---+------+



In [8]:
df_outer = df1.join(df2, on="Name", how="outer")
print("Outer Join:")
df_outer.show()

Outer Join:
+-------+----+------+----+------+
|   Name| Age|Gender| Age|Gender|
+-------+----+------+----+------+
|  David|  19|     M|NULL|  NULL|
|Jasmine|NULL|  NULL|  50|     F|
|   John|NULL|  NULL|  35|     M|
|   Mark|  25|     M|NULL|  NULL|
|Shirley|  22|     F|  22|     F|
+-------+----+------+----+------+



In [9]:
common_cols = [col for col in df1.columns if col in df2.columns]

df_natural = df1.join(df2, on=common_cols, how="inner")
print("Natural join:")
df_natural.show()

Natural join:
+-------+---+------+
|   Name|Age|Gender|
+-------+---+------+
|Shirley| 22|     F|
+-------+---+------+



In [10]:
spark.stop()